### RAG with minsearch

In [8]:
import requests
import minsearch

from openai import OpenAI

In [5]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    for doc in course["documents"]:
        doc["course"] = course["course"]
        documents.append(doc)
        

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [9]:
oa_client = OpenAI()

In [11]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )

    return results
    

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT to answer the QUESTION.

QUESTION: ```{question}```

CONTEXT:

```{context}```
""".strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestin: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context)
    return prompt


In [14]:
def llm(prompt):
    response = oa_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content
    

In [15]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer
    

In [16]:
rag("how to run Kafka?")

'To run Kafka, you need to execute the following command in your project directory:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your JAR file. Make sure you have built your project beforehand and have the necessary dependencies in place.'

### RAG with Vector Search

In [21]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from fastembed import TextEmbedding

In [19]:
qd_client = QdrantClient("http://localhost:6333")

In [28]:
supported_models = TextEmbedding.list_supported_models()
for m in supported_models:
    if m["dim"] == 512:
        print(m["model"])

BAAI/bge-small-zh-v1.5
Qdrant/clip-ViT-B-32-text
jinaai/jina-embeddings-v2-small-en


In [29]:
embedding_dim = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [31]:
collection_name = "zoomcamp-faq"

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=embedding_dim,
        distance=models.Distance.COSINE
    )
)

True

In [33]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
points = []

for i, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"]
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)
    

In [37]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [38]:
def vector_search(query, course="data-engineering-zoomcamp"):
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )

    return [point.payload for point in query_points.points]
    

In [40]:
def rag(query):
    search_results = vector_search(query, "data-engineering-zoomcamp")
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [41]:
rag("how to run Kafka?")

'To run Kafka, ensure that your Kafka broker docker container is running. You can confirm this by using the command `docker ps`. If the Kafka broker is not running, navigate to the directory containing the docker-compose YAML file and run `docker compose up -d` to start all instances.\n\nAdditionally, when running Java producer or consumer scripts, ensure that the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` in your script is set to the correct server URL, and that your cluster key and secrets in `Secrets.java` are updated with the correct values.\n\nFor Python scripts, create a virtual environment, install required packages using `pip install -r requirements.txt`, and activate the environment before running your scripts.'